In [15]:
import numpy as np
import pandas as pd

## Import Metastatic Tissue Dataset

In [16]:
import torch
from torch import nn
from torch.utils.data import Dataset
import h5py
from tqdm.notebook import tqdm
import torch.nn.functional as F

In [4]:
def load_data(filename):
    with h5py.File(filename, "r") as f:
        # List all groups
        a_group_key = list(f.keys())[0]

        # Get the data
        data = torch.from_numpy(f[a_group_key][:])
        return data
    
training_images = load_data("./pcam/training_split.h5")
print(training_images.shape)

training_labels = load_data("./Labels/Labels/camelyonpatch_level_2_split_train_y.h5").squeeze()
print(training_labels.shape)

torch.Size([262144, 96, 96, 3])
torch.Size([262144])


In [146]:
class TissueCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.convd = nn.Conv2d(in_channels=3, out_channels=1, kernel_size = (4,4))
        self.maxPool = nn.MaxPool2d(kernel_size = (2,2), stride = 1)
        self.Relu = nn.ReLU()
        self.linear = nn.Linear(8464, 2)
        self.soft = nn.Softmax()

    def forward(self, x):
        x = x.reshape(1,3,96,96)
        x = x.type(torch.FloatTensor)
#         print(x.shape)
        out = self.convd(x)
#         print(out.shape)
        out = self.maxPool(out)
#         print(out.shape)
        out = self.Relu(out)
#         print(out.shape)
        out = torch.flatten(out)
#         print(out.shape)
        out = self.linear(out)
#         print(out.shape)
        out = self.soft(out)
#         print(out.shape)
        return out

### Data Cleaning

In [147]:
model = TissueCNN()
print(model)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, weight_decay=0)
# batch_size = 64
# train_loader = torch.utils.data.DataLoader(train_set, batch_size, sampler = train_sampler)
# val_loader = torch.utils.data.DataLoader(train_set, batch_size, sampler = val_sampler)
# test_loader = torch.utils.data.DataLoader(test_set, batch_size)

TissueCNN(
  (convd): Conv2d(3, 1, kernel_size=(4, 4), stride=(1, 1))
  (maxPool): MaxPool2d(kernel_size=(2, 2), stride=1, padding=0, dilation=1, ceil_mode=False)
  (Relu): ReLU()
  (linear): Linear(in_features=8464, out_features=2, bias=True)
  (soft): Softmax(dim=None)
)


In [148]:
def train_network(model, training_images, labels, criterion, optimizer, nepoch=100):
    try:
        for epoch in tqdm(range(nepoch)):
            print('EPOCH %d'%epoch)
            total_loss = 0
            count = 0
#             print("labels")
#             print(labels.shape)
            ##
            for inp, label in zip(training_images, labels):
                optimizer.zero_grad()
                outputs = model.forward(inp)
#                 print("Outputs {} {} label {}".format(outputs, outputs.shape, label))
#                 print("Here")
#                 print(label.item())
#                 print(outputs)
#                 print(outputs.shape)
#                 print(label.shape)
#                 print("Here")
                label = label.unsqueeze(0)
                label = label.type(torch.LongTensor)
                outputs = outputs.unsqueeze(0)
#                 print(outputs.shape)
#                 print(label.shape)
                loss = criterion(outputs, label)
                loss.backward()
                optimizer.step()
                total_loss += loss.item()
                count += 1
            print('{:>12s} {:>7.5f}'.format('Train loss:', total_loss/count))
            print()
    except KeyboardInterrupt:
        print('Exiting from training early')
    return

In [ ]:

train_network(model, training_images, training_labels.squeeze(), criterion, optimizer)

  0%|          | 0/100 [00:00<?, ?it/s]

EPOCH 0


<ipython-input-146-5c61db49eb37>:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  out = self.soft(out)


 Train loss: 0.81326

EPOCH 1
 Train loss: 0.81326

EPOCH 2
 Train loss: 0.81326

EPOCH 3
 Train loss: 0.81326

EPOCH 4
 Train loss: 0.81326

EPOCH 5
 Train loss: 0.81326

EPOCH 6
 Train loss: 0.81326

EPOCH 7
 Train loss: 0.81326

EPOCH 8
 Train loss: 0.81326

EPOCH 9
 Train loss: 0.81326

EPOCH 10
 Train loss: 0.81326

EPOCH 11
